# 카피라이팅 도구 만들기

## 테스트

In [1]:
print("안녕하세요")

안녕하세요


## (1) 모델 불러오기

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# 환경변수 불러오기
load_dotenv()

# Model 생성
model = ChatOpenAI(
    model="gpt-4o-mini",  # 비용 gpt-4.1-nano < gpt-4o-mini < gpt-4
    temperature=0.5       # 창의성 정도(0~1 사이의 값. 1에 가까울수록 창의적인 답변)
)

# 사용자 질문에 대한 응답
question = """
너는 콘텐츠 기획자야.
관람객을 끌어들일 홍보 문구를 1~2문장으로 써 줘.
이번 콘텐츠는 5분 분량의 영상에 태블릿 PC 환경에서의 챗봇을 결합한 창의적인인 작품이야.
말투는 감성적이고 따뜻했으면 좋겠어.
"""

response = model.invoke(question)
print(response)

content='"당신의 마음을 따뜻하게 감싸줄 5분의 이야기에 초대합니다. 태블릿 PC 속에서 펼쳐지는 챗봇과의 특별한 만남, 그 속에서 당신의 상상력이 날개를 달고 날아오를 것입니다."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 91, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-Bbndh7HlonIaRnWCyn2m0KmRv9g2j', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--44af7aec-25a9-4f20-bf10-f804a4f12588-0' usage_metadata={'input_tokens': 91, 'output_tokens': 61, 'total_tokens': 152, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [3]:
# 내용만 출력
print(response.content)

"당신의 마음을 따뜻하게 감싸줄 5분의 이야기에 초대합니다. 태블릿 PC 속에서 펼쳐지는 챗봇과의 특별한 만남, 그 속에서 당신의 상상력이 날개를 달고 날아오를 것입니다."


## (2) OutputParser 기능

In [4]:
from langchain_core.output_parsers import StrOutputParser

# OutputParser 생성
output_parser = StrOutputParser()

In [5]:
# Chain 생성
chain = model | output_parser 

# 사용자 질문에 대한 응답
response = chain.invoke(question)
print(response)

# question을  model에 넣는다. 나온 결과를 output_parser에 넣는다.

"당신의 마음을 따뜻하게 감싸줄 5분의 순간, 태블릿 PC 속에서 만나는 챗봇의 이야기를 통해 새로운 감성을 경험해보세요. 함께 떠나는 작은 여행, 당신의 소중한 시간을 더욱 특별하게 만들어줄 것입니다."


## (3) Prompt 기능

In [6]:
from langchain_core.prompts import PromptTemplate

# PromptTemplate 정의
template = """
너는 콘텐츠 기획자야.
다음 정보에 맞춰서 관람객을 끌어들일 창의적인 공연 홍보 문구를 {count}개 만들어줘.

- 공연 설명: {description}
- 문구 길이: {length}
- 말투: {tone}
"""
# Prompt 생성
prompt = PromptTemplate.from_template(template)
# 사용자 입력에 대한 프롬프트 결과 (위의 template에 사용자의 값을 입력해 주세요_확인용)
prompt_with_variables = prompt.invoke(
    {
        "description": "오래된 연인과의 기억, 시간이 지나도 사라지지 않는 감정들을 무대 위에 풀어낸 작품",
        "length": "1~2줄",
        "tone": "감성적, 따뜻함",
        "count": 5
    }
)
print(prompt_with_variables)

text='\n너는 콘텐츠 기획자야.\n다음 정보에 맞춰서 관람객을 끌어들일 창의적인 공연 홍보 문구를 5개 만들어줘.\n\n- 공연 설명: 오래된 연인과의 기억, 시간이 지나도 사라지지 않는 감정들을 무대 위에 풀어낸 작품\n- 문구 길이: 1~2줄\n- 말투: 감성적, 따뜻함\n'


In [7]:
# Chain 생성
chain = prompt | model | output_parser 

# 사용자 입력에 대한 응답
response = chain.invoke(
    {
        "description": "태블릿 PC로 감상, 챗봇을 접목한 영상 작품",
        "length": "1~2줄",
        "tone": "감성적, 따뜻함",
        "count": 3
    }
)
print(response)

1. "당신의 마음을 따뜻하게 감싸줄, 태블릿 속 챗봇과 함께하는 새로운 영상의 세계로 초대합니다."

2. "디지털 감성과 따뜻한 대화가 어우러진 특별한 경험, 태블릿으로 만나는 감동의 순간을 놓치지 마세요."

3. "챗봇과 함께하는 감성적인 영상 여행, 태블릿 속에서 펼쳐지는 따뜻한 이야기를 만나보세요."
